In [1]:
import pandas as pd
import gurobipy as gb
from gurobipy import *
import numpy as np
from vizlib import *

In [316]:
avail = pd.read_csv('availability.csv')
num_weeks = len(avail)
weeks = list(range(0,num_weeks))

path = list(range(0,5))
path_i = path
res = list(range(5,9))
res_i = list(range(0,4))
all = path + res

b_target = np.array([12/52, 12/52, 12/52, 2/52, 12/52, 12/52, 12/52, 12/52, 12/52]) * num_weeks
n_target = b_target

n_rp_freq = n_target[res]/len(path)
n_rp_target = np.tile(n_rp_freq.reshape((-1,1)), len(path))

b_rp_freq = b_target[res]/len(path)
b_rp_target = np.tile(b_rp_freq.reshape((-1,1)), len(path))

m = gb.Model()

# Decision variables
b = m.addMVar(avail.shape, name='biopsies', vtype=GRB.BINARY)
n = m.addMVar(avail.shape, name='necropsies', vtype=GRB.BINARY)
c = m.addMVar(avail.shape, vtype=GRB.BINARY, name='combined')
m.addConstrs((c[:,a] == b[:,a] + n[:,a] for a in all))

# Each duty requires one pathologist
m.addConstrs((quicksum(b[w,path]) == 1 for w in weeks), name='b_p_req')
m.addConstrs((quicksum(n[w,path]) == 1 for w in weeks), name='n_p_req')

# Each duty can have a max of one resident
m.addConstrs((quicksum(b[w,res]) <= 1 for w in weeks), name='b_r_max')
m.addConstrs((quicksum(n[w,res]) <= 1 for w in weeks), name='n_r_max')

# Cannot work both duties or during time off
m.addConstrs((b[w,a] + n[w,a] <= avail.iloc[w,a] for w in weeks for a in all), name='availability')

# Penalize double booking - except in the case of N/B which is ideal
# Weights based on number of weeks waiting for N labs to come back and then do them
# May need to be modified depending on who is doing the microscope samples
nn_first = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)
nn_second = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)

bn_first = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)
bn_second = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)

bb_first = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)
bb_second = m.addVars(num_weeks-1, len(all), vtype=GRB.BINARY)

m.addConstrs((nn_first[w,a] + nn_second[w,a] == n[w,a] + n[w+1,a] for w in weeks[:-2] for a in all))
m.addConstrs((bn_first[w,a] + bn_second[w,a] == b[w,a] + n[w+1,a] for w in weeks[:-2] for a in all))
m.addConstrs((bb_first[w,a] + bb_second[w,a] == b[w,a] + b[w+1,a] for w in weeks[:-2] for a in all))

double_cost = 3*nn_second.sum() + 2*bn_second.sum() + bb_second.sum()

# # Penalize triple booking - eventually favor triple booking with 3rd year resident
# Define combined schedule variable
first_two_triple = m.addVars(num_weeks-2, len(all), ub=2, vtype=GRB.INTEGER)
triples = m.addVars(num_weeks-2, len(all), vtype=GRB.BINARY)

m.addConstrs((first_two_triple[w,a] + triples[w,a] == quicksum(c[w:w+3,a]) for w in weeks[:-2] for a in all))

triple_cost = triples.sum()

# # Any streak containing N should be followed with no scheduled and availability REWARD: TO-DO?

# # Even pairing penalty
n_pairs = m.addVars(len(res), len(path), num_weeks, vtype=GRB.INTEGER)
n_pairs_sum = m.addVars(len(res), len(path), vtype=GRB.INTEGER)
n_pairs_under = m.addVars(len(res), len(path))
m.addConstrs((n_pairs[ir,ip,w] <= n[w,p] for w in weeks for ip,p in enumerate(path) for ir,_ in enumerate(res)))
m.addConstrs((n_pairs[ir,ip,w] <= n[w,r] for w in weeks for ip,_ in enumerate(path) for ir,r in enumerate(res)))
m.addConstrs((n_pairs[ir,ip,w] >= n[w,r]+n[w,p]-1 for w in weeks for ip,p in enumerate(path) for ir,r in enumerate(res)))
m.addConstrs((n_pairs_sum[r,p] == sum(n_pairs[r,p,w] for w in weeks) for p in path_i for r in res_i))
m.addConstrs((n_pairs_under[r,p] >= n_rp_target[r,p] - n_pairs_sum[r,p] for p in path_i for r in res_i))

b_pairs = m.addVars(len(res), len(path), num_weeks, vtype=GRB.INTEGER)
b_pairs_sum = m.addVars(len(res), len(path), vtype=GRB.INTEGER)
b_pairs_under = m.addVars(len(res), len(path))
m.addConstrs((b_pairs[ir,ip,w] <= b[w,p] for w in weeks for ip,p in enumerate(path) for ir,_ in enumerate(res)))
m.addConstrs((b_pairs[ir,ip,w] <= b[w,r] for w in weeks for ip,_ in enumerate(path) for ir,r in enumerate(res)))
m.addConstrs((b_pairs[ir,ip,w] >= b[w,r]+b[w,p]-1 for w in weeks for ip,p in enumerate(path) for ir,r in enumerate(res)))
m.addConstrs((b_pairs_sum[r,p] == sum(b_pairs[r,p,w] for w in weeks) for p in path_i for r in res_i))
m.addConstrs((b_pairs_under[r,p] >= b_rp_target[r,p] - b_pairs_sum[r,p] for p in path_i for r in res_i))

pair_cost = n_pairs_under.sum() + b_pairs_under.sum()

# Penalize mismatch of number of duties per employee
b_diff = m.addVars(len(all), name='biopsy_target_difference', lb=-float('inf'))
b_under = m.addVars(len(all), name='biopsy_target_under')
b_over = m.addVars(len(all), name='biopsy_target_over')
m.addConstrs((b_diff[a] == quicksum(n[w,a] for w in weeks) - b_target[a] for a in all), name='b_diff')
m.addConstrs((b_under[a] >= -b_diff[a] for a in all), name='b_under')
m.addConstrs((b_over[a] >= b_diff[a] for a in all), name='b_over')

n_diff = m.addVars(len(all), name='necropsy_target_difference', lb=-float('inf'))
n_under = m.addVars(len(all), name='necropsy_target_under')
n_over = m.addVars(len(all), name='necropsy_target_over')
m.addConstrs((n_diff[a] == quicksum(b[w,a] for w in weeks) - n_target[a] for a in all), name='n_diff')
m.addConstrs((n_under[a] >= -n_diff[a] for a in all), name='n_under')
m.addConstrs((n_over[a] >= n_diff[a] for a in all), name='n_over')

duty_balance_cost = n_under.sum() + n_over.sum() + b_under.sum() + b_over.sum()

# Objective function
m.setObjective(duty_balance_cost + 10*triple_cost + double_cost + pair_cost, gb.GRB.MINIMIZE) # 
m.optimize()

# Visualize the schedule with avail dataframe, b matrix and n matrix
viz_schedule(avail, b, n)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 3800 rows, 2963 columns and 11538 nonzeros
Model fingerprint: 0x6690c52a
Variable types: 94 continuous, 2869 integer (1818 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 2e+00]
  RHS range        [8e-01, 5e+00]
Found heuristic solution: objective 298.9230714
Presolve removed 1983 rows and 1469 columns
Presolve time: 0.03s
Presolved: 1817 rows, 1494 columns, 5675 nonzeros
Found heuristic solution: objective 272.9230723
Variable types: 36 continuous, 1458 integer (1393 binary)

Root relaxation: objective 1.369231e+01, 1615 iterations, 0.12 seconds (0.07 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.69231    0  528  272.9

,NM,MP,SA,LKC,DG,MC,AK,CC,KB
0,,B,N,-,,,-,N,B
1,B,-,-,-,N,B,N,,-
2,,N,-,-,B,,B,N,-
3,N,-,-,-,B,,,,B
4,B,N,-,-,,N,B,-,
5,,B,,-,N,,-,-,N
6,N,-,B,-,-,N,,,B
7,B,-,N,-,-,,N,B,-
8,,B,-,N,-,N,B,,-
9,N,,-,-,B,B,-,N,


In [323]:
pair_cost.getValue(), double_cost.getValue(), triple_cost.getValue(), duty_balance_cost.getValue()

(4.846153846153847, 3.0, 1.0, 5.230769230769226)